In [17]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
import scipy.optimize as spo

In [18]:
%matplotlib ipympl

In [19]:
# Sample geometry
t = 3*10**-6
tErr = 0.01*10**-6
d = 1.5*10**-3
D = 6.5*10**-3
rhoGeoErr = -8.5*(d/D)**4
RhGeoErr = -4.4*(d/D)**2

In [32]:
print d/D
print rhoGeoErr
print RhGeoErr

0.230769230769
-0.0241062987991
-0.234319526627


In [20]:
# Sheet resistance measurements
V1 = np.array([21.68, 21.68, 21.68, 21.68,
              6.297, 6.296, 6.295, 6.297]) * 10**-3
# Hall resistance measurements
V2 = np.array([15.389, 15.388, -15.393, -15.393]) * 10**-3
I = 3 * 10**-3

# Sheet resistance
R1 = V1/I
# Hall resistance
R2 = V2/I

# Lab data
----------------------------------------------------------------------

## n-type GaAs (t=3$\mu$m):

#### Measuring the sheet resistance, $R_{sh}$

| Current I, (mA) |Voltage, $V_{ij,kl}$| Value, (mV) | Resistance ($\Omega$)
| --------------- |:------------:|:------------:|:------------:|
|       3         | $V_{14,23}$  | 21.68 | 7.23
|       3         | $V_{41,32}$  | 21.68 | 7.23
|       3         | $V_{23,14}$  | 21.68 | 7.23
|       3         | $V_{32,41}$  | 21.68 | 7.23
|       3         | $V_{12,43}$  | 6.297 | 2.10
|       3         | $V_{21,34}$  | 6.296 | 2.10
|       3         | $V_{43,12}$  | 6.295 | 2.10
|       3         | $V_{34,21}$  | 6.297 | 2.10
|       3         | $V_{42,31}$  | 15.393| 

#### Measuring the Hall Resistance
| Current I, (mA) |Voltage, $V_{ij,kl}$| Value, (mV) | Resistance ($\Omega$)
| --------------- |:------------:|:------------:|:------------:|
|       3         | $V_{13,24}$  | 15.489 | 5.13
|       3         | $V_{31,42}$  | 15.488 | 5.13
|       3         | $V_{24,31}$  | -15.493 | 5.13
|       3         | $V_{42,13}$  | -15.493 | 5.13

In [21]:
# Van der Pauw method for measuring sheet resistivity
Ra = np.average(R1[4:])
RaErr = 0.005*np.sqrt(4)
Rb = np.average(R1[0:4])
RbErr = 0.005*np.sqrt(4)
ratio = 21.68/15.393
ratioErr = ratio * np.sqrt( (RaErr/Ra)**2 + (RbErr/Rb)**2 )

In [22]:
# Equation to solve for value of f
def f(y, x=1.408):
    return np.cosh((x-1)/(x+1) * np.log(2)/y) - 0.5*np.exp(np.log(2)/y)

In [23]:
# Solution for f
x0 = ratio
z = spo.brentq(f, 0.8, 0.99, (x0))

In [24]:
# Calculating sheet resistance and resistivity
R_sh = (np.pi / np.log(2)) * ((Ra+Rb)/2) * z
R_shErr = np.sqrt(RaErr**2 + RbErr**2)
rho = R_sh * t
rhoMeasErr = rho * np.sqrt( (R_shErr/R_sh)**2 + (tErr/t)**2 )
rhoErr = np.sqrt((rhoMeasErr)**2 + (rhoGeoErr*rho)**2)
print 'rho = {:.2e} +/- {:.2e} Ohm m'.format(rho, rhoErr)
print 'sigma = ' + str(rho**-1)+ "(Ohm m)^-1"
print 'R_sh = {:.2f} +/- {:.2e}'.format(R_sh, R_shErr)

rho = 6.28e-05 +/- 1.53e-06 Ohm m
sigma = 15933.1066261(Ohm m)^-1
R_sh = 20.92 +/- 1.41e-02


#### $R_{H}$ : Measuring the Hall coefficient, for fixed magnetic field strength

Positive Field, Positive Current

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 100.2   | 1.449
|       2.70         | 100.2   | 1.313
|       2.40         | 100.2   | 1.171
|       2.10         | 100.2   | 1.026
|       1.80         | 100.2   | 0.879
|       1.50         | 100.2   | 0.733
|       1.20         | 96.9   | 0.586
|       0.90         | 100.2   |0.440
|       0.60         | 97.1   |0.294
|       0.30         | 100.2   |0.148



Positive Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 100.2   | -1.449
|       2.70         | 100.2   | -1.313
|       2.40         | 100.2   | -1.172
|       2.10         | 100.2   | -1.025
|       1.80         | 100.2   | -0.879
|       1.50         | 100.2   | -0.732
|       1.20         | 100.2   | -0.586
|       0.90         | 100.2   | -0.439
|       0.60         | 100.2   | -0.293
|       0.30         | 96.8   | -0.147


Negative Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -100.3   | 1.464
|       2.70         | -100.3   | 1.318
|       2.40         | -100.3   | 1.167
|       2.10         | -95.7    | 1.021
|       1.80         | -100.3   | 0.875
|       1.50         | -100.3   | 0.730
|       1.20         | -100.3   | 0.584
|       0.90         | -100.3   | 0.439
|       0.60         | -95.6   | 0.294
|       0.30         | -95.4   | 0.158


Negative Field, Positive current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -100.3   | -1.459
|       2.70         | -100.3   | -1.313
|       2.40         | -100.3   | -1.167
|       2.10         | -100.3   | -1.021
|       1.80         | -100.3   | -0.875
|       1.50         | -100.3   | -0.729
|       1.20         | -100.3   | -0.583
|       0.90         | -100.3   | -0.438
|       0.60         | -100.3   | -0.292
|       0.30         | -95.8   | -0.147

-------------------------------------------


Positive Field, Positive Current

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 150.1   | 2.263
|       2.70         | 150.1   | 2.037
|       2.40         | 150.1   | 1.810
|       2.10         | 150.2   | 1.585
|       1.80         | 150.2   | 1.358
|       1.50         | 150.3   | 1.132
|       1.20         | 150.3   | 0.906
|       0.90         | 150.3   |0.680
|       0.60         | 146.8   |0.454
|       0.30         | 150.3   |0.228

Positive Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 150.1   | -2.263
|       2.70         | 150.1   | -2.037
|       2.40         | 150.1   | -1.811
|       2.10         | 150.1   | -1.584
|       1.80         | 150.1   | -1.358
|       1.50         | 150.1   | -1.132
|       1.20         | 150.1   | -0.906
|       0.90         | 150.1   | -0.680
|       0.60         | 150.0   | -0.453
|       0.30         | 150.1   | -0.227

Negative Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -150.4   | 2.252
|       2.70         | -150.4   | 2.028
|       2.40         | -150.4   | 1.807
|       2.10         | -150.3   | 1.577
|       1.80         | -150.4   | 1.352
|       1.50         | -150.3   | 1.127
|       1.20         | -150.3   | 0.901
|       0.90         | -150.3   | 0.677
|       0.60         | -150.2   | 0.452
|       0.30         | -145.4   | 0.227

Negative Field, Positive current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -150.5   | -2.253
|       2.70         | -150.8   | -2.027
|       2.40         | -150.8   | -1.803
|       2.10         | -150.8   | -1.577
|       1.80         | -150.8   | -1.352
|       1.50         | -150.8   | -1.127
|       1.20         | -150.8   | -0.902
|       0.90         | -150.8   | -0.676
|       0.60         | -150.8   | -0.451
|       0.30         | -150.8   | -0.226


-------------------------------------------

Positive Field, Positive Current

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 203.0   | 3.105
|       2.70         | 203.1   | 2.795
|       2.40         | 203.1   | 2.482
|       2.10         | 203.1   | 2.173
|       1.80         | 203.1   | 1.862
|       1.50         | 203.1   | 1.552
|       1.20         | 203.1   | 1.242
|       0.90         | 203.1   | 0.932
|       0.60         | 203.1   | 0.622
|       0.30         | 200.6   | 0.305

Positive Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 203.1   | -3.094
|       2.70         | 203.1   | -2.785
|       2.40         | 203.1   | - 2.476 
|       2.10         | 203.1   | - 2.167
|       1.80         | 203.1   | -1.857
|       1.50         | 203.1   | -1.548
|       1.20         | 203.1   | -1.238
|       0.90         | 203.1   | -0.929
|       0.60         | 203.1   | -0.621
|       0.30         | 200.0   | -0.311

Negative Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -200.7   | 3.076
|       2.70         | -200.5   | 2.769
|       2.40         | -200.5   | 2.461
|       2.10         | -200.3   | 2.153
|       1.80         | -200.3   | 1.846
|       1.50         | -200.0   | 1.539
|       1.20         | -199.8   | 1.232
|       0.90         | -199.7   | 0.924
|       0.60         | -199.5   | 0.617
|       0.30         | -198.5   | 0.324

Negative Field, Positive current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -201.9   | -3.076
|       2.70         | -201.7   | -2.770
|       2.40         | -201.6   | -2.461
|       2.10         | -201.5   | -2.154
|       1.80         | -201.4   | -1.846
|       1.50         | -201.3   | -1.539
|       1.20         | -201.2   | -1.231
|       0.90         | -201.1   | -0.927
|       0.60         | -200.9   | -0.616
|       0.30         | -198.6   | -0.309


-------------------------------------------

Positive Field, Positive Current

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 248.9   | 3.894
|       2.70         | 248.7   | 3.314
|       2.40         | 248.6   | 3.110
|       2.10         | 248.6   | 2.772
|       1.80         | 249.5   | 2.334
|       1.50         | 248.4   | 1.945
|       1.20         | 249.3   | 1.555
|       0.90         | 249.3   | 1.167
|       0.60         | 249.3   | 0.779
|       0.30         | 251.1   | 0.397
|       2.70         | 248.7   | 3.500
|       2.40         | 248.6   | 3.112
|       2.10         | 248.6   | 2.723
|       1.80         | 249.5   | 2.340
|       1.50         | 248.4   | 1.946
|       1.20         | 249.3   | 1.556
|       0.90         | 249.3   | 1.168
|       0.60         | 249.3   | 0.778
|       0.30         | 251.1   | 0.390

Positive Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 249.9   | -3.891
|       2.70         | 249.8   | -3.501
|       2.40         | 249.7   | -3.112
|       2.10         | 249.6   | -2.724
|       1.80         | 249.5   | -2.335
|       1.50         | 249.4   | -1.946
|       1.20         | 249.3   | -1.557
|       0.90         | 249.2   | -1.167
|       0.60         | 251.2   | -0.778
|       0.30         | 251.1   | -0.390

Negative Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -249.7   | 3.860
|       2.70         | -249.7   | 3.480
|       2.40         | -249.7   | 3.093
|       2.10         | -249.6   | 2.707
|       1.80         | -249.7   | 2.320
|       1.50         | -249.7   | 1.934
|       1.20         | -248.6   | 1.547
|       0.90         | -249.6   | 1.156
|       0.60         | -249.6   | 0.775
|       0.30         | -249.7   | 0.404

Negative Field, Positive current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -250.3   | -3.868
|       2.70         | -249.7   | -3.480
|       2.40         | -249.6   | -3.094
|       2.10         | -249.5   | -2.707
|       1.80         | -249.4   | -2.320
|       1.50         | -249.3   | -1.934
|       1.20         | -249.3   | -1.547
|       0.90         | -249.2   | -1.161
|       0.60         | -249.1   | -0.774
|       0.30         | -249.7   | -0.388



-------------------------------------------

Positive Field, Positive Current

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 301   | 4.683
|       2.70         | 301   | 4.214
|       2.40         | 301   | 3.748
|       2.10         | 301 | 3.279
|       1.80         | 301   | 2.812
|       1.50         | 301   | 2.343
|       1.20         | 301     | 1.874
|       0.90         | 301   | 1.405
|       0.60         | 301   | 0.938
|       0.30         | 299.8   | 0.470
|       2.70         | 302   | 4.241
|       2.40         | 302   | 3.796
|       2.10         | 302 | 3.296
|       1.80         | 302   | 2.827
|       1.50         | 302   | 2.356
|       1.20         | 302     | 1.885
|       0.90         | 302   | 1.413
|       0.60         | 302   | 0.943
|       0.30         | 303   | 0.472

Positive Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | 302   | -4.685
|       2.70         | 302   | -4.216
|       2.40         | 302   | -3.748
|       2.10         | 302   | -3.280
|       1.80         | 302   | -2.811
|       1.50         | 302   | -2.343
|       1.20         | 302   | -1.875
|       0.90         | 302   | -1.407
|       0.60         | 301   | -0.938
|       0.30         | 301   | -0.471

Negative Field, Negative current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -301   | 4.664
|       2.70         | -301   | 4.198
|       2.40         | -301   | 3.731
|       2.10         | -301   | 3.264
|       1.80         | -301   | 2.799
|       1.50         | -301   | 2.332
|       1.20         | -301   | 1.866
|       0.90         | -301   | 1.401
|       0.60         | -301   | 0.935
|       0.30         | -302   | 0.484

Negative Field, Positive current:

| Current I, (mA) |Magnetic Field B, (mT) | Voltage, V_H (mV)
| --------------- |:------------:|:------------:|
|       3.00         | -301   | -4.664
|       2.70         | -302   | -4.198
|       2.40         | -301!   | -3.730
|       2.10         | -301   | -3.265
|       1.80         | -301   | -2.798
|       1.50         | -301   | -2.333
|       1.20         | -301   | -1.866
|       0.90         | -301   | -1.400
|       0.60         | -301   | -0.934
|       0.30         | -302   | -0.468

In [25]:
I = np.arange(0.3,3.3,0.3)
B = np.arange(0.1, 0.35, 0.05)
Berr = B*0.01
V = np.empty((5,4,10))
V[0] = [[0.148, 0.294, 0.440, 0.586, 0.733,
             0.879, 1.026, 1.171, 1.313, 1.449],
        [-0.147, -0.293, -0.439, -0.586, -0.732,
                -0.879, -1.025, -1.172, -1.313, -1.449],
        [0.158, 0.294, 0.439, 0.584, 0.730,
                0.875, 1.021, 1.167, 1.318, 1.464],
        [-0.147, -0.292, -0.438, -0.583, -0.729,
                -0.875, -1.021, -1.167, -1.313, -1.459]]
V[1] = [[0.228, 0.454, 0.680, 0.906, 1.132,
                1.358, 1.585, 1.810, 2.037, 2.263],
        [-0.227, -0.453, -0.680, -0.906, -1.132,
                -1.358, -1.584, -1.811, -2.037, -2.263],
        [0.227, 0.452, 0.677, 0.901, 1.127,
                1.352, 1.577, 1.807, 2.028, 2.252],
        [-0.226, -0.451, -0.676, -0.902, -1.127,
            -1.352, -1.477, -1.803, -2.027, -2.253]]
V[2] = [[0.305, 0.622, 0.932, 1.242, 1.552,
            1.862, 2.173, 2.482, 2.795, 3.105],
        [-0.311, -0.621, -0.929, -1.238, -1.548,
            -1.857, -2.167, -2.476, -2.785, -3.094],
        [0.324, 0.617, 0.924, 1.232, 1.539,
            1.846, 2.153, 2.461, 2.769, 3.076],
        [-0.309, -0.61, -0.927, -1.231, -1.539,
            -1.846, -2.154, -2.461, -2.770, -3.076]]
V[3] = [[0.397, 0.779, 1.167, 1.555, 1.945,
            2.334, 2.772, 3.110, 3.314, 3.894],
        [-0.390, -0.778, -1.167, -1.557, -1.946,
            -2.335, -2.724, -3.112, -3.501, -3.891],
        [0.404, 0.775, 1.156, 1.547, 1.934,
            2.320, 2.707, 3.093, 3.480, 3.860],
        [-0.388, -0.774, -1.161, -1.547, -1.934,
            -2.320, -2.707, -3.094, -3.480, -3.868]]
V[4] = [[0.470, 0.938, 1.405, 1.874, 2.343,
            2.812, 3.279, 3.748, 4.214, 4.638],
        [-0.471, -0.938, -1.407, -1.875, -2.343,
            -2.811, -3.280, -3.748, -4.216, -4.685],
        [0.484, 0.935, 1.401, 1.866, 2.332,
            2.799, 3.264, 3.731, 4.198, 4.664],
        [-0.468, -0.934, -1.400, -1.866, -2.333,
            -2.798, -3.265, -3.730, -4.198, -4.664]]
V_100=np.sum(V[0][1:3:2] - V[0][0:2:2], axis=0)/2
V_150=np.sum(V[1][1:3:2] - V[1][0:2:2], axis=0)/2
V_200=np.sum(V[2][1:3:2] - V[2][0:2:2], axis=0)/2
V_250=np.sum(V[3][1:3:2] - V[3][0:2:2], axis=0)/2
V_300=np.sum(V[4][1:3:2] - V[4][0:2:2], axis=0)/2
Vt = np.array([V_100, V_150, V_200, V_250, V_300]).T
Verr = 0.005

### Transverse (Hall) Resistance

In [26]:
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
ax2.set_title('A plot of Transverse Voltage against Current, \n at various magnetic field strengths')

line_100, = ax2.plot(I, V_100, label="100mT")
line_150, = ax2.plot(I, V_150, label="150mT")
line_200, = ax2.plot(I, V_200, label="200mT")
line_250, = ax2.plot(I, V_250, label="250mT")
line_300, = ax2.plot(I, V_300, label="300mT")
ax2.set_xlabel("Current I / (mA)")
ax2.set_ylabel("Hall Output Voltage / (mV)")
ax2.legend()

FigureCanvasNbAgg()

In [27]:
fig3 = plt.figure()
ax3 = fig3.add_subplot(111)
ax3.set_title('A plot of Hall Resistance against magnetic field strength \n under constant current supply')

Vplot = Vt[0::3]
R = (Vplot.T/I[0::3]).T
RErr = R * np.sqrt((Verr/Vplot)**2)
line2_03 = ax3.errorbar(B, R[0], label="0.3mA", yerr=RErr[0], xerr=Berr[0])
line2_12 = ax3.errorbar(B, R[1], label="1.2mA", yerr=RErr[1], xerr=Berr[1])
line2_21 = ax3.errorbar(B, R[2], label="2.1mA", yerr=RErr[2], xerr=Berr[2])
line2_30 = ax3.errorbar(B, R[3], label="3.0mA", yerr=RErr[3], xerr=Berr[3])
ax3.set_xlabel("Magnetic induction (T)")
ax3.set_ylabel("Hall Resistance / ($\Omega$)")
ax3.legend()

FigureCanvasNbAgg()

In [28]:
VB = Vt/B
VBerr = VB * np.sqrt((Verr/Vt)**2 + (Berr/B)**2)

In [29]:
def y(x, A, B):
    return A*x + B
fit_data = [curve_fit(y, I, v, sigma=e, absolute_sigma=True) for v,e in zip(VB.T,VBerr.T)]
#fit_data = [curve_fit(y, I, v) for v in VB.T]
grad = np.array([i[0][0] for i in fit_data])
intercept = np.array([i[0][1] for i in fit_data])
perr = np.array([np.sqrt(np.diag(i[1])) for i in fit_data])
#gradErr = np.sqrt( (VBerr/VB)**2 + (Berr/B)**2)
gradErr = np.sqrt((perr.T[0])**2 + (RhGeoErr*grad)**2)

In [30]:
mu = grad/rho * t
muErr = mu * np.sqrt( (gradErr/grad)**2 + (rhoErr/rho)**2 + (tErr/t)**2  )
n = (grad*t*-1.67*10**-19)**-1
nErr = n * np.sqrt( (gradErr/grad)**2 + (tErr/t)**2)
print "Rh = {:.3e} +/- {:.2e}".format(np.average(grad), np.average(gradErr))
print "u = {:.3f} +/- {:.3f}".format(np.average(mu), np.average(np.abs(muErr)))
print "n = {:.3e} +/- {:.3e}".format(np.average(n), np.average(nErr))

Rh = -5.081e+00 +/- 1.19e+00
u = -0.243 +/- 0.057
n = 3.931e+23 +/- 9.214e+22


$\mu = 2500 \pm 570{cm}^{2}V^{-1}s^{-1}$

$n = 3.95 \pm 0.92x10^{17} {cm}^{-3}$